In [1]:
from sklearn.datasets import load_wine

wine = load_wine()

In [2]:
print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [3]:
x = wine.data
y = wine.target

In [4]:
x.shape, y.shape

((178, 13), (178,))

In [9]:
import pandas as pd

pd.DataFrame(x, columns=wine.feature_names).describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


In [16]:
wine.target_names

array(['class_0', 'class_1', 'class_2'], dtype='<U7')

In [18]:
# 1. 전처리
# 1) train/test split
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=2022)

# 2) scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

# 3. one-hot encoding
from tensorflow.keras.utils import to_categorical

len_y = len(set(train_y))
train_y = to_categorical(train_y, len_y)
test_y = to_categorical(test_y, len_y)

In [19]:
train_y.shape, test_y.shape

((142, 3), (36, 3))

In [40]:
# 2. 모델링
from tensorflow import keras

# 1) 세션 클리어
keras.backend.clear_session()

# 2) 레이어 생성
il = keras.layers.Input(shape=(13,))
ol = keras.layers.Dense(3, activation='softmax')(il)

# 3) 모델 생성
model = keras.models.Model(il, ol)

# 4) 모델 컴파일
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# 5) 모델 요약
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense (Dense)               (None, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [41]:
# 3. 모델 학습
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=5,
                   verbose=1,
                   restore_best_weights=True)

model.fit(train_x, train_y, epochs=100, validation_split=0.1, callbacks=[es], verbose=1)

Epoch 1/100
4/4 [==============================] - 1s 53ms/step - loss: 1.1354 - accuracy: 0.4173 - val_loss: 1.0833 - val_accuracy: 0.4667
Epoch 2/100
4/4 [==============================] - 0s 9ms/step - loss: 1.1297 - accuracy: 0.4252 - val_loss: 1.0785 - val_accuracy: 0.4667
Epoch 3/100
4/4 [==============================] - 0s 10ms/step - loss: 1.1243 - accuracy: 0.4409 - val_loss: 1.0738 - val_accuracy: 0.4667
Epoch 4/100
4/4 [==============================] - 0s 10ms/step - loss: 1.1187 - accuracy: 0.4252 - val_loss: 1.0688 - val_accuracy: 0.4667
Epoch 5/100
4/4 [==============================] - 0s 12ms/step - loss: 1.1136 - accuracy: 0.4252 - val_loss: 1.0643 - val_accuracy: 0.4667
Epoch 6/100
4/4 [==============================] - 0s 10ms/step - loss: 1.1081 - accuracy: 0.4252 - val_loss: 1.0596 - val_accuracy: 0.5333
Epoch 7/100
4/4 [==============================] - 0s 10ms/step - loss: 1.1028 - accuracy: 0.4331 - val_loss: 1.0549 - val_accuracy: 0.5333
Epoch 8/100
4/4 [====

In [42]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(test_x)
accuracy_score(y_pred.argmax(axis=1), test_y.argmax(axis=1))

0.8611111111111112

In [37]:
# 2. 모델링 (히든레이어 있는 버전)
from tensorflow import keras

# 1) 세션 클리어
keras.backend.clear_session()

# 2) 레이어 생성
il = keras.layers.Input(shape=(13,))
h1 = keras.layers.Dense(64, activation='relu')(il)
h2 = keras.layers.Dense(64, activation='relu')(h1)
ol = keras.layers.Dense(3, activation='softmax')(h2)

# 3) 모델 생성
model = keras.models.Model(il, ol)

# 4) 모델 컴파일
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# 5) 모델 요약
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense (Dense)               (None, 64)                896       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                                 
Total params: 5,251
Trainable params: 5,251
Non-trainable params: 0
_________________________________________________________________


In [38]:
# 3. 모델 학습
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=5,
                   verbose=1,
                   restore_best_weights=True)

model.fit(train_x, train_y, epochs=100, validation_split=0.1, callbacks=[es], verbose=1)

Epoch 1/100
4/4 [==============================] - 1s 56ms/step - loss: 1.0763 - accuracy: 0.4724 - val_loss: 1.0330 - val_accuracy: 0.6667
Epoch 2/100
4/4 [==============================] - 0s 10ms/step - loss: 1.0220 - accuracy: 0.6850 - val_loss: 0.9750 - val_accuracy: 0.8000
Epoch 3/100
4/4 [==============================] - 0s 11ms/step - loss: 0.9711 - accuracy: 0.7244 - val_loss: 0.9255 - val_accuracy: 0.8667
Epoch 4/100
4/4 [==============================] - 0s 10ms/step - loss: 0.9245 - accuracy: 0.7874 - val_loss: 0.8784 - val_accuracy: 0.9333
Epoch 5/100
4/4 [==============================] - 0s 11ms/step - loss: 0.8785 - accuracy: 0.8268 - val_loss: 0.8306 - val_accuracy: 0.9333
Epoch 6/100
4/4 [==============================] - 0s 10ms/step - loss: 0.8315 - accuracy: 0.8740 - val_loss: 0.7838 - val_accuracy: 1.0000
Epoch 7/100
4/4 [==============================] - 0s 14ms/step - loss: 0.7836 - accuracy: 0.9291 - val_loss: 0.7373 - val_accuracy: 1.0000
Epoch 8/100
4/4 [===

In [36]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(test_x)
accuracy_score(y_pred.argmax(axis=1), test_y.argmax(axis=1))

1.0